# Review Rec Bot

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

# Arguments

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Review Rec Bot - Yelp Review Rec Bot",
    RUN_NAME="015_revamp_synthetic_eval",
    RUN_DESCRIPTION="""
# Revamp the evaluation questions generation to better match a recommendation chatbot's job

## Changelog
- Change the retrieval_question_gen_query and response_question_gen_query
- Increase number of sampled documents from 20 to 30
- Enable query expansion
""",
    TESTING=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=False,
    RECREATE_RETRIEVAL_EVAL_DATASET=True,
    RECREATE_RESPONSE_EVAL_DATASET=True,
    DEBUG=False,
)

ARGS

{
  "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
  "RUN_NAME": "015_revamp_synthetic_eval",
  "RUN_DESCRIPTION": "\n# Revamp the evaluation questions generation to better match a recommendation chatbot's job\n\n## Changelog\n- Change the retrieval_question_gen_query and response_question_gen_query\n- Increase number of sampled documents from 20 to 30\n- Enable query expansion\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_INDEX": false,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true
}

# Load config

In [7]:
from src.run.cfg import RunConfig

In [8]:
cfg = RunConfig()
cfg.init(ARGS)

2024-08-09 17:08:09.947 | INFO     | src.run.cfg:init:139 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/dvquys/.phoenix
2024-08-09 17:08:14.517 | INFO     | src.run.cfg:init:157 - Setting up MLflow experiment Review Rec Bot - Yelp Review Rec Bot - run 015_revamp_synthetic_eval...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-08-09 17:08:14.739 | INFO     | src.run.cfg:init:166 - Notebook-generated artifacts are persisted at data/015_revamp_synthetic_eval


In [9]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Review Rec Bot - Yelp Review Rec Bot",
    "RUN_NAME": "015_revamp_synthetic_eval",
    "RUN_DESCRIPTION": "\n# Revamp the evaluation questions generation to better match a recommendation chatbot's job\n\n## Changelog\n- Change the retrieval_question_gen_query and response_question_gen_query\n- Increase number of sampled documents from 20 to 30\n- Enable query expansion\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_INDEX": false,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true
  },
  "app_name": "review_rec_bot",
  "storage_context_persist_dp": "data/009_hybrid_retriever/storage_context",
  "db_collection": "review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5",
  "notebook_cache_dp": "data/015_revamp_synthetic_eval",
  "data_fp": "../data/yelp_dataset/sample/sample_100_biz/denom_review.parquet",
  "llm_c

## Set up logger to collect additional info

In [10]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [11]:
data = pd.read_parquet(cfg.data_fp)
data = data.assign(
    biz_categories=lambda df: df['biz_categories'].str.split(', '),
    date=lambda df: df['date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
)
logger.info(f"[COLLECT] {len(data)=}")
data.iloc[0]

2024-08-09 17:08:14.900 | INFO     | __main__:<module>:6 - [COLLECT] len(data)=5240


review_id                                      L0jv8c2FbpWSlfNC6bbUEA
user_id                                        bFPdtzu11Oi0f92EAcjqmg
business_id                                    IDtLPgUrqorrpqSLdfMhZQ
review_stars                                                        5
useful                                                              0
funny                                                               0
cool                                                                0
text                What a great addition to the Funk Zone!  Grab ...
date                                              2016-10-13T22:50:47
biz_name                                         Helena Avenue Bakery
biz_address                                     131 Anacapa St, Ste C
biz_city                                                Santa Barbara
biz_state                                                          CA
biz_postal_code                                                 93101
biz_latitude        

In [12]:
input_data = data
if ARGS.TESTING:
    input_data = data[:20]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-08-09 17:08:14.929 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=5240


In [13]:
input_data.columns

Index(['review_id', 'user_id', 'business_id', 'review_stars', 'useful',
       'funny', 'cool', 'text', 'date', 'biz_name', 'biz_address', 'biz_city',
       'biz_state', 'biz_postal_code', 'biz_latitude', 'biz_longitude',
       'biz_stars', 'biz_review_count', 'biz_is_open', 'biz_attributes',
       'biz_categories', 'biz_hours'],
      dtype='object')

# Prepare documents

In [14]:
from llama_index.core import Document

documents = []
embedding_visible_metadata = ["review_stars", "biz_name", "biz_address", "biz_city", "biz_state", "biz_categories"]
excluded_embed_metadata_keys = [k for k in input_data.columns if k not in embedding_visible_metadata]

for i, row in tqdm(input_data.iterrows(), total=len(input_data)):
    record = row.to_dict()
    text = record['text']
    metadata = {k: v for k, v in record.items() if k not in ('text')}
    
    doc = Document(
        text=text,
        metadata=metadata,
        excluded_embed_metadata_keys=excluded_embed_metadata_keys,
        excluded_llm_metadata_keys=excluded_embed_metadata_keys
    )
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

  0%|          | 0/5240 [00:00<?, ?it/s]

2024-08-09 17:08:15.487 | INFO     | __main__:<module>:20 - [COLLECT] len(documents)=5240


#### Check document embedding text

In [15]:
from llama_index.core.schema import MetadataMode

In [16]:
document = documents[0]
print(document.get_content(metadata_mode=MetadataMode.EMBED))

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.


# Set up LLM

In [17]:
llm, embed_model = cfg.setup_llm()

Some weights of BertModel were not initialized from the model checkpoint at Snowflake/snowflake-arctic-embed-m-v1.5 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "togetherai",
  "llm_model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
  "embedding_provider": "huggingface",
  "embedding_model_name": "Snowflake/snowflake-arctic-embed-m-v1.5",
  "embedding_model_dim": 768,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [19]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [20]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [21]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    enable_hybrid=False,
    prefer_grpc=True
)

2024-08-09 17:08:24.697 | INFO     | src.run.orchestrator:setup_db:37 - Use existing Qdrant collection: review_rec_bot__009_hybrid_retriever__huggingface__Snowflake_snowflake_arctic_embed_m_v1_5
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [22]:
import time
import pickle
from multiprocessing import set_start_method
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.node_parser import SemanticSplitterNodeParser
# from llama_index.core.node_parser import SentenceSplitter

In [23]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

# chunker = SentenceSplitter
# chunker_cfg = {
#     "chunk_size": 512,
#     "chunk_overlap": 10
# }

In [24]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading Storage Context from {cfg.storage_context_persist_dp}...")
    docstore = SimpleDocumentStore.from_persist_dir(persist_dir=cfg.storage_context_persist_dp)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    nodes = list(docstore.docs.values())
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache

    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # TODO: I can get the preprocessing completed much quicker if I push the initial steps through a multiprocessing loop and then separately create the embeddings using the built-in batching already provided in the SentenceTransformer encode method.
    # Ref: https://github.com/run-llama/llama_index/issues/10104#issuecomment-1899401584
    # Currently setting num_workers leads to code simple hang
    # Ref: https://github.com/run-llama/llama_index/issues/10104
    # num_workers = os.cpu_count() - 1
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # set_start_method("spawn", force=True)  # it hangs without this line
    logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers, show_progress=True)

    # Define Docstore as an abstraction on top of nodes to easily manage (e.g. get node by id)
    docstore = SimpleDocumentStore()
    await docstore.async_add_documents(nodes)
    storage_context = StorageContext.from_defaults(
        docstore=docstore,
        vector_store=vector_store
    )
    logger.info(f"Persisting Storage Context to {cfg.storage_context_persist_dp}...")
    storage_context.persist(cfg.storage_context_persist_dp)

t1 = time.perf_counter()

2024-08-09 17:08:24.822 | INFO     | __main__:<module>:7 - Loading Storage Context from data/009_hybrid_retriever/storage_context...


In [25]:
logger.info(f"Indexing {len(documents)} documents into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-08-09 17:08:32.459 | INFO     | __main__:<module>:1 - Indexing 5240 documents into VectorStoreIndex took 8s
2024-08-09 17:08:32.459 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=10219


# Analyze Chunks

In [26]:
for i, node in enumerate(nodes[:5]):
    print(f"\n\n==========Node {i+1}============")
    print(node.metadata)
    print(node.get_text())



==========Node 1============
{'review_id': 'L0jv8c2FbpWSlfNC6bbUEA', 'user_id': 'bFPdtzu11Oi0f92EAcjqmg', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-10-13T22:50:47', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly':

# Construct Retriever

In [27]:
from llama_index.core.retrievers import VectorIndexRetriever

In [28]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 10,
  "retrieval_dense_top_k": 5,
  "retrieval_sparse_top_k": 15,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 5,
  "rerank_model_name": "BAAI/bge-reranker-large"
}


In [29]:
# configure retriever
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
vector_retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=cfg.retrieval_cfg.retrieval_dense_top_k,
    # sparse_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
)

In [30]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=cfg.retrieval_cfg.retrieval_sparse_top_k,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

bm25_retrieval_results = bm25_retriever.retrieve("Nashville, grocery")
for result in bm25_retrieval_results:
    display_source_node(result, source_length=1000, show_source_metadata=True)

DEBUG [bm25s] Building index from IDs objects


**Node ID:** db6b2d69-c359-4927-8849-ff1581e2eada<br>**Similarity:** 3.1722183227539062<br>**Text:** A good grocery store, just requires a car to get to (or bus). This used to be somewhere they shuttled local university students from Penn to come get groceries; not sure if they still do that.

There's almost always a good deal to be had, fresh produce and meat, and feels like a truly suburban grocery store. Lines are long at times, but the only real problem is the distance from the city if you don't have a car.<br>**Metadata:** {'review_id': 'hKTTE9mOS1tXI6gDbDU_Mw', 'user_id': 'nnwBdqGHIAJQ5QX9lHOtrQ', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 3, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2009-01-20T16:11:51', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 8ec77c6a-6c2a-4b53-b928-259a4b5ccf8b<br>**Similarity:** 3.0762240886688232<br>**Text:** This grocery store SUCKS!  But it's the closest one to my house, so I go there for things that I need fast.  The produce is the worst possible.  I don't think the prices are very good, but then again it seems that all grocery stores in Philly are outrageously priced.  Employees are the slowest moving in the world!<br>**Metadata:** {'review_id': 'krw5FlAWXS16NkkqViK-wg', 'user_id': 'B-d6QdtkkoLHCyIyuT6y4A', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2011-05-12T20:29:04', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

**Node ID:** 2020bdc4-4d5b-419d-9581-81cd7076823c<br>**Similarity:** 3.0317909717559814<br>**Text:** Free parking garage.  Eat in grocery store or deli type restaurant....everything is by the pound.<br>**Metadata:** {'review_id': 'vQxUaNanrYGZ_X3qCiIC3g', 'user_id': 'btAiOvMIfAbP5Sc5fAR8zA', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-07-08T18:07:49', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** ea7d973e-a656-4e3a-93c6-bd64ac553b3f<br>**Similarity:** 2.975971221923828<br>**Text:** Not the best grocery store in the world but the prices are reasonable. My cashier kept literally screaming for help.<br>**Metadata:** {'review_id': 'n8JpZwGLs3BwBzIPGlz7BQ', 'user_id': 'oD1WErpp65gbk29ErcDsLw', 'business_id': '-4dYswJy7SPcbcERvitmIg', 'review_stars': 2, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-10-12T15:53:42', 'biz_name': 'Pathmark', 'biz_address': '3021 Grays Ferry Ave', 'biz_city': 'Philadelphia', 'biz_state': 'PA', 'biz_postal_code': '19146', 'biz_latitude': 39.9404026, 'biz_longitude': -75.1932966, 'biz_stars': 2.5, 'biz_review_count': 34, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': None, 'Ambience': None, 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': None, 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': None, 'GoodForMeal': None, 'HappyHour': None, 'HasTV': None, 'Music': None, 'NoiseLevel': None, 'OutdoorSeating': None, 'RestaurantsAttire': None, 'RestaurantsDelivery': None, 'RestaurantsGoodForGroups': None, 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': None, 'RestaurantsTableService': None, 'RestaurantsTakeOut': None, 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': None}, 'biz_categories': ['Food', 'Grocery'], 'biz_hours': {'Friday': '6:0-1:0', 'Monday': '6:0-1:0', 'Saturday': '6:0-1:0', 'Sunday': '6:0-1:0', 'Thursday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'Wednesday': '6:0-1:0'}}<br>

...

**Node ID:** 3ab49015-e92a-4891-8557-c9f42c886837<br>**Similarity:** 2.6478774547576904<br>**Text:** If you're looking for great tasting, high quality Italian food then this is the place for you. They make their own bread that is to die for. They have the best manicotti that I've ever had. Yum! There isn't a single bad thing on the menu. They also offer a unique Italian grocery selection with imported and domestic Italian products that are very hard to find at regular grocery stores. Support small business and give this place a try!<br>**Metadata:** {'review_id': '7jSEUBirxk_ghx8OI9myQg', 'user_id': 'DIXNEOmHO6D-zOM0X9fNEg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 1, 'date': '2014-01-03T22:13:26', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** b1c55b3a-ce09-46d4-ae03-795999f88bc8<br>**Similarity:** 2.636214256286621<br>**Text:** Really neat concept!  There is a small gourmet grocery. A soup and sandwich station, a salad station and a combination beer, wine and coffee bar.  Loved the frozen rose!<br>**Metadata:** {'review_id': 'b4aG7Vq86gHcAIt99FiJrA', 'user_id': 'HBjWfVa3BaRxFAH6TDc1Nw', 'business_id': 'lk9IwjZXqUMqqOhM774DtQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2017-09-23T02:17:54', 'biz_name': 'Caviar & Bananas', 'biz_address': '2031 Broadway', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37203', 'biz_latitude': 36.1483712, 'biz_longitude': -86.7988947, 'biz_stars': 3.5, 'biz_review_count': 159, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'beer_and_wine'", 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': True, 'casual': True}", 'BYOB': None, 'BestNights': "{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}", 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': 'False', 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': 'False', 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': True, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'True', 'HasTV': 'False', 'Music': "{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}", 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': "u'no'", 'WheelchairAccessible': 'True', 'WiFi': "'free'"}, 'biz_categories': ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners'], 'biz_hours': {'Friday': '7:0-17:0', 'Monday': '7:0-17:0', 'Saturday': '7:0-17:0', 'Sunday': '7:0-17:0', 'Thursday': '7:0-17:0', 'Tuesday': '7:0-17:0', 'Wednesday': '7:0-17:0'}}<br>

**Node ID:** e002734f-0e43-47e9-84a7-1b6a6910f4f6<br>**Similarity:** 2.5576841831207275<br>**Text:** A must stop in Nashville<br>**Metadata:** {'review_id': 'BbJRG7HkGx0S3SuvvpVvIQ', 'user_id': '26nVu2FZTrYt3PEMXLCR0A', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2013-09-28T04:48:16', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

**Node ID:** 139ef666-1ca5-4606-8799-25705d5f50aa<br>**Similarity:** 2.4747650623321533<br>**Text:** Wow!!!! Perfect and delicious. As good as the best delis in NYC or Boston. Fast service and nice folks. Plus the little grocery of hard to find Italian specialties !!! Fantastic !! Yum !! Come hungry !!<br>**Metadata:** {'review_id': 'Wxw8_PKy98_Fo3yLKcwTnA', 'user_id': 'HnnDiOQETwAEMfGZfLJ9gg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2016-06-20T16:36:12', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

**Node ID:** 973a0309-7588-44d7-ae3d-0e8f8d5fc159<br>**Similarity:** 2.4747650623321533<br>**Text:** Wow!!!! Perfect and delicious. As good as the best delis in NYC or Boston. Fast service and nice folks. Plus the little grocery of hard to find Italian specialties !!! Fantastic !! Yum !! Come hungry !!<br>**Metadata:** {'review_id': 'wd5V8nsdvBkmi6ra-bh1hg', 'user_id': 'HnnDiOQETwAEMfGZfLJ9gg', 'business_id': '0bPLkL0QhhPO5kt1_EXmNQ', 'review_stars': 5, 'useful': 2, 'funny': 0, 'cool': 0, 'date': '2016-03-31T17:40:24', 'biz_name': "Zio's Italian Market", 'biz_address': '2575 E Bay Dr', 'biz_city': 'Largo', 'biz_state': 'FL', 'biz_postal_code': '33771', 'biz_latitude': 27.9161159, 'biz_longitude': -82.7604608, 'biz_stars': 4.5, 'biz_review_count': 100, 'biz_is_open': 0, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': 'False', 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': None, 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': False}", 'HappyHour': None, 'HasTV': 'True', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'False', 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'False', 'RestaurantsPriceRange2': '1', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Delis', 'Italian', 'Bakeries', 'Restaurants'], 'biz_hours': {'Friday': '10:0-20:0', 'Monday': '10:0-18:0', 'Saturday': '10:0-20:0', 'Sunday': None, 'Thursday': '10:0-20:0', 'Tuesday': '10:0-20:0', 'Wednesday': '10:0-20:0'}}<br>

In [31]:
# test_query = nodes[0].text
test_query = "Received confirmation of my order at 5 pm"
vector_results = vector_retriever.retrieve(test_query)
bm25_results = bm25_retriever.retrieve(test_query)
assert bm25_results[0].id_ == vector_results[0].id_, "Different vector_retriever nodes vs bm25_retriever nodes"

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Extract the key concepts and entities as keywords. This technique can generate multiple strings instead of just one. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  string 1 = "cold brew coffee", string 2 = "hanoi". Do not modify the keyword, keep the keyword exactly in the input query. Return each output string on a new line.
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

# query_gen_prompt = """
# You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
# You should apply different techniques to create new strings. Here are some example techniques:
# - Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
# - Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."
# - Technique 3 - Increase recall: Paraphase the original question into a new question but use different synonyms or broader concepts. Example input query: "How to find some good local grocery stores near Changi Singapore?" -> Expected output query: "Where to find a good mart selling Singaporean stuffs to the east of Singapore?"

# Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2, third technique 3. If run of of techniques then re-iterate from the start.

# Return one string on each line, related to the input query.

# Only return the strings. Never include the chosen technique.

# Input Query: {query}\n
# New strings:\n
# """

query_gen_prompt = """
You are a helpful assistant that expands an input query into new strings that aim to increase the recall of an information retrieval system. The strings can be queries or paragraphs or sentences.
You should apply different techniques to create new strings. Here are some example techniques:
- Technique 1 - Optimize for full-text search: Rephrase the input query to contain only important keywords. Remove all stopwords and low information words. Example input query: "What are some places to enjoy cold brew coffee in Hanoi?" -> Expected output:  "cold brew coffee hanoi"
- Technique 2 - Optimize for similarity-based vector retrieval: Create a fake user review that should contain the answer for the question. Example input query: "What are some good Pho restaurants in Singapore?" -> Expected output query: "I found So Pho offerring a variety of choices to enjoy not Pho but some other Vietnamese dishes like bun cha. The price is reasonable."

Generate at least {num_queries} new strings by iterating over the technique in order. For example, your first generated string should always use technique 1, second technique 2. If run of of techniques then re-iterate from the start.

Return one string on each line, related to the input query.

Only return the strings. Never include the chosen technique.

Input Query: {query}\n
New strings:\n
"""

from llama_index.llms.openai import OpenAI
llm = OpenAI(model=cfg.eval_cfg.response_eval_llm_model, **cfg.eval_cfg.response_eval_llm_model_config)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    llm=llm,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
    num_queries=2,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt
)

## Test retrieval

In [33]:
query = "What are some places to enjoy cold brew coffee"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

Generated queries:
cold brew coffee places
"I recently visited a cozy café that specializes in cold brew coffee. The atmosphere was perfect for relaxing, and they had a fantastic selection of cold brew options that really hit the spot!"


**Node ID:** 17d515f7-ee7b-4ad8-ba5e-7ade77e44922<br>**Similarity:** 0.03333333333333333<br>**Text:** This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee.<br>

review_stars: 5
biz_name: Helena Avenue Bakery
biz_address: 131 Anacapa St, Ste C
biz_city: Santa Barbara
biz_state: CA
biz_categories: ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries']

This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee.




**Node ID:** eb0d0583-a323-4295-86c4-256d98f63110<br>**Similarity:** 0.03306010928961749<br>**Text:** I would recommend this place!<br>

review_stars: 4
biz_name: BAP
biz_address: 1224 South St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Korean', 'Restaurants']

I would recommend this place!




...

review_stars: 4
biz_name: Wawa
biz_address: 3604 Chestnut St
biz_city: Philadelphia
biz_state: PA
biz_categories: ['Restaurants', 'Automotive', 'Delis', 'Gas Stations', 'Food', 'Coffee & Tea', 'Sandwiches', 'Convenience Stores']

I'm a fan of dark roast but it's usually not available. My advice brew different flavors of coffee and keep it hot.




**Node ID:** 31fc1577-f72b-4a44-88d3-f98f751666e1<br>**Similarity:** 0.029857397504456328<br>**Text:** I am in love! This is my new favorite brew place! The best part about it, the variety! I was lucky to go on a pint night, the vibe was fun, and everyone was there loving these good brews. I go to GABF every year and this is a nice mini taste of a varietal of beers. I had their Kolsch, it was crisp and cold as it should be. I am not always the biggest hoppy fan, but the DFG IPA was clean, light and very fresh tasting. It didn't kill my taste buds as I enjoyed the Pumkin beer (best one I have had all fall and winter) next!<br>

review_stars: 5
biz_name: Bier Brewery and Tap Room
biz_address: 5133 E 65th St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Food', 'Beer', 'Wine & Spirits', 'Breweries']

I am in love! This is my new favorite brew place! The best part about it, the variety! I was lucky to go on a pint night, the vibe was fun, and everyone was there loving these good brews. I go to GABF every year and this is a nice mini taste of a varietal of beers. I had their Kolsch, it was crisp and cold as it should be. I am not always the biggest hoppy fan, but the DFG IPA was clean, light and very fresh tasting. It didn't kill my taste buds as I enjoyed the Pumkin beer (best one I have had all fall and winter) next!




**Node ID:** d07fa28e-c8b1-4bd9-89ab-e2cb3e850c18<br>**Similarity:** 0.02967032967032967<br>**Text:** Best neighbors ever!!

If I would wish for a dream neighbor it would be Caviar & Bananas. 
You know, that neighbor that is never in your business but always there when you need them! 
The one that always have a cup of sugar and 2 eggs. Well, this place is it, they fulfill so many of my needs!

Need a healthy but still delicious lunch? Check!
Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

Best neighbors ever!!

If I would wish for a dream neighbor it would be Caviar & Bananas. 
You know, that neighbor that is never in your business but always there when you need them! 
The one that always have a cup of sugar and 2 eggs. Well, this place is it, they fulfill so many of my needs!

Need a healthy but still delicious lunch? Check!
Need a bottle of wine? Check!
Need a cold brew coffee with just a splash of almond milk? Check!




## Retrieval Evaluation

### Synthetic

In [34]:
from src.run.eval import RetrievalEvaluator

In [35]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-08-09 17:09:07.356 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:36 - Creating new retrieval eval dataset at data/015_revamp_synthetic_eval/retrieval_synthetic_eval_dataset.json...
2024-08-09 17:09:07.357 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:40 - Sampling 30 nodes for retrieval evaluation...
2024-08-09 17:09:07.374 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:67 - Creating new synthetic retrieval eval dataset...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:25<00:00,  1.19it/s]
2024-08-09 17:09:32.537 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:74 - Persisting synthetic retrieval eval dataset to data/015_revamp_synthetic_eval/retrieval_synthetic_eval_dataset.json...


In [36]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'dbfb4988-5308-4097-8f58-b6606a3e845f': 'What are some good parking options available near LAPLCO Blvd & Manhattan?',
 '9d2fa648-f5a2-40e1-81c2-d5020cf0fb2c': 'What are some reputable shipping companies known for handling packages without damage?',
 'de83b07a-743a-44e3-b3d2-7286f3897345': 'What are some affordable parking options in Midtown Nashville?',
 '0dfb1efe-e023-42a4-9540-e1e05fed0fa7': 'What are some businesses or locations that have received poor reviews for customer service?',
 'edfd8ea6-5424-4c1c-b01a-6dc67ba9aca0': 'What are some ice cream shops that offer unique flavors and a nostalgic atmosphere?',
 '6d9dc768-30c5-46bf-8975-920250d5bebd': 'What are some popular ice cream shops in Nashville that offer waffle cones?',
 'ab8c899e-7c30-4b0a-b2da-d742415933c3': 'What are some sushi restaurants that offer a la carte options instead of tasting menus?',
 '0693c761-676f-45a7-963e-cfca4b8ce418': 'What are some cafes known for their breakfast sandwiches and high-quality coffee?',
 

In [37]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

Generated queries:
parking options LAPLCO Blvd Manhattan
"I recently visited the area near LAPLCO Blvd and Manhattan and found several great parking options. There’s a lot right across the street that’s affordable and convenient, plus some street parking available if you arrive early."
Generated queries:
reputable shipping companies package handling damage-free
"I recently used SafeShip for my deliveries, and I was impressed by their careful handling. My packages arrived without a scratch, and their customer service was top-notch!"
Generated queries:
affordable parking options midtown nashville
"I recently visited Midtown Nashville and found several affordable parking spots. The parking garage on 21st Avenue was a great option, and I also noticed street parking available near the restaurants."
Generated queries:
poor customer service reviews businesses locations
"I visited a few places recently, and I was really disappointed with the service. One restaurant had rude staff, and another 

In [38]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_10_retrieval_eval,0.433333,0.186005,0.043333,0.433333,0.186005,0.053596


In [39]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
0,What are some good parking options available n...,[e398cffc-eb02-42e3-923a-604ae06c4889],"[59e9e8ed-c6fb-4434-acc2-67f33dfd2069, e398cff...",1.0,0.500000,0.1,1.0,0.500000,0.138862
1,What are some reputable shipping companies kno...,[03e033d4-2831-439e-8ea6-a4946a41916a],"[57ee580f-56d1-41a5-8aae-2753dc8138d5, 3377961...",1.0,0.333333,0.1,1.0,0.333333,0.110046
2,What are some affordable parking options in Mi...,[7bb1304d-9638-4b37-89b3-b8c852ffeff2],"[5a4f24a2-23da-4fc1-8660-0ef6b846129a, eb0d058...",1.0,0.142857,0.1,1.0,0.142857,0.073364
3,What are some businesses or locations that hav...,[70f23ccc-9c0d-478c-a3fa-602529082d56],"[3377961f-0b68-44c9-ac77-d8941156e582, 411b6f6...",0.0,0.000000,0.0,0.0,0.000000,0.000000
4,What are some ice cream shops that offer uniqu...,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[acf34ef2-1ab1-4705-a205-d2fa322c9749, bc4fad1...",0.0,0.000000,0.0,0.0,0.000000,0.000000
5,What are some popular ice cream shops in Nashv...,[fd39867d-dddb-4ae8-abd2-9e0422dce67a],"[f5afbee4-fbcb-45ee-821a-f39d08b8dfa8, 67d8275...",1.0,0.250000,0.1,1.0,0.250000,0.094788
6,What are some sushi restaurants that offer a l...,[5c86dd25-9c62-4117-ad76-87823a901d02],"[c5c24971-3fb0-4e27-a564-01188d07ee3a, 72a5572...",0.0,0.000000,0.0,0.0,0.000000,0.000000
7,What are some cafes known for their breakfast ...,[07042f7e-af55-4f78-9c1b-5259eaa0cd11],"[f3710ad8-06f6-4c34-8ee6-437ef55e4dbe, 09c2819...",1.0,0.142857,0.1,1.0,0.142857,0.073364
8,What are some financial institutions known for...,[2b20d945-b172-4a11-b82f-8c3238d42a16],"[77c48c1b-cde7-40d9-8d72-985e3ee10cc5, 2d38afc...",0.0,0.000000,0.0,0.0,0.000000,0.000000
9,What are some restaurants known for their reli...,[955626ac-20b8-422e-9562-302188e37c11],"[3377961f-0b68-44c9-ac77-d8941156e582, 18bd79e...",0.0,0.000000,0.0,0.0,0.000000,0.000000


#### Error Analysis

In [40]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
with pd.option_context('display.max_colwidth', 400):
    display(retrieval_eval_irrelevance_df)

,query,expected_ids,retrieved_ids,hit_rate,mrr,precision,recall,ap,ndcg
3,What are some businesses or locations that have received poor reviews for customer service?,[70f23ccc-9c0d-478c-a3fa-602529082d56],"[3377961f-0b68-44c9-ac77-d8941156e582, 411b6f66-9951-4032-9c1b-6e9045069b6f, 44946e7b-630e-4c75-b4d6-f28516bceafc, d59bbaed-472a-42dc-8fdd-d38b5a9d0242, 928aa9e5-86f8-4315-9405-4c8038b64ea4, 86d888ca-a8ae-4f44-addd-4f7c23f3c036, 55cee61e-a0fc-4e85-8ae1-b13861f30f71, 181636f8-10a0-46f3-9ecc-234350f8d398, 1621c153-c0ee-4528-b1d7-55e7a5db255f, 1d1c801f-bcf9-44c7-860b-299295930934]",0.0,0.0,0.0,0.0,0.0,0.0
4,What are some ice cream shops that offer unique flavors and a nostalgic atmosphere?,[f89a64e7-2b86-4333-89fa-11a18dc7d897],"[acf34ef2-1ab1-4705-a205-d2fa322c9749, bc4fad12-8d88-4bcc-9b71-d0a6d550cdcc, d15b604d-f289-442b-9619-09260e0c29a6, a67f63c6-a802-413c-806a-16735b28f347, 49c3f474-ef09-4bf0-a248-8219c80e5bc0, d739ed4d-7a9b-4957-baea-0355bab54b36, eb0d0583-a323-4295-86c4-256d98f63110, 43a57307-7784-4df7-9100-c32f6ea01414, 330dfd87-96d9-4925-a906-4df81dbe542b, 3832bae5-b6d2-4bca-860a-d928fb323bb7]",0.0,0.0,0.0,0.0,0.0,0.0
6,What are some sushi restaurants that offer a la carte options instead of tasting menus?,[5c86dd25-9c62-4117-ad76-87823a901d02],"[c5c24971-3fb0-4e27-a564-01188d07ee3a, 72a55725-0f5d-42a8-854f-7bf3bd4675cc, 1ddd93b1-7fc0-4489-997f-82f33b41f4c8, 586c6d7b-9d62-40a9-973d-0801ac4c416f, 34a7e616-51eb-4866-a596-11a4b0f97913, d66fab88-b654-4ee3-a8a8-73769b127a47, 4f131715-050b-4802-9c56-4e068c2b139e, b5c0aacb-6bd9-46e7-8d91-1480660398db, dd887aa3-34b3-47dc-bc1d-f57dbf12d53a, 4df22824-9acb-407f-8e9e-426c5c0aa57e]",0.0,0.0,0.0,0.0,0.0,0.0
8,What are some financial institutions known for better customer service and support?,[2b20d945-b172-4a11-b82f-8c3238d42a16],"[77c48c1b-cde7-40d9-8d72-985e3ee10cc5, 2d38afc3-1b57-4108-9471-e3fdbab1fa25, ad52a0fb-9725-40f6-83f6-2c6097377d12, bfe518c9-84e1-4092-924c-8a2395af1cf0, c63f28f3-6ea6-46fc-ac18-5c0c5eb5263f, eb74777c-c297-4d8b-b430-e97c2b4bf1e0, a201a9b6-0205-40ea-994a-cc44498a4146, 4caedf20-1d96-431b-8fd6-b906e9b1bea7, 00bceaee-8175-41fa-b4fd-90342250cab3, a590ad78-72a4-46eb-a52a-93fb1df24b27]",0.0,0.0,0.0,0.0,0.0,0.0
9,What are some restaurants known for their reliable service and consistent food quality?,[955626ac-20b8-422e-9562-302188e37c11],"[3377961f-0b68-44c9-ac77-d8941156e582, 18bd79e3-060d-4032-a6f6-e9f239dc33b2, 7ba10220-0c7a-44f4-9e52-0202803df53e, 713747c0-ed84-4dde-9163-f23f137fdc33, c5c24971-3fb0-4e27-a564-01188d07ee3a, 181636f8-10a0-46f3-9ecc-234350f8d398, dbc69608-fbaa-4405-9db3-7891d1a2cba3, e4470636-8d62-489a-ba26-8d90551a0813, 02cd763b-dd78-460e-83a0-d71960c11cfa, da7b4cf5-0648-4958-9397-0eda17f9c616]",0.0,0.0,0.0,0.0,0.0,0.0
12,What are some popular shopping malls or centers where friends can enjoy a day out together?,[08dfdbc3-ca3f-4be3-a547-cca56fa68455],"[bc4fad12-8d88-4bcc-9b71-d0a6d550cdcc, ec2ff10b-4784-4443-955e-00bb69a0a620, 86d888ca-a8ae-4f44-addd-4f7c23f3c036, 59e9e8ed-c6fb-4434-acc2-67f33dfd2069, 965da779-3a3d-4bc7-8e7b-27e73902980c, 4b58086c-1fea-4eec-adbf-2801c299761c, f3a3b291-0948-40b0-9cf6-0da6f46cc229, d347a88c-3ea2-4f13-9a1b-27d1a176d643, 98b56c36-0097-4ff4-b703-9f4796b120c2, c6c2b0db-b8fb-4d22-b54e-9892d8cca043]",0.0,0.0,0.0,0.0,0.0,0.0
13,What are some cafes or restaurants that offer healthy breakfast options?,[c92d191f-fdbb-4442-98cb-a823ead8df82],"[6f9f2e39-c895-4d31-8cd5-a8ac2c2d1810, d6d04b18-dcd5-4a18-9973-8c6f4fe9e88c, ba1198bb-7d75-426a-8e43-036300f3dd45, 3f79a723-f810-4143-af50-82cd6d1317bc, bb6908e5-9a9b-4f11-8131-52507f41c86b, a8a4e7b0-11fc-4a3b-80a6-9791f90e452d, 923a03c6-1a15-44a5-ba25-03917e37a4d7, 32c72efa-4362-40f6-9653-2863bbdd03b5, 1aa39de5-e1e6-4a67-8be8-c390e7eb5d20, fa52e4c7-bbb8-4aa9-98ca-f0810817b45c]",0.0,0.0,0.0,0.0,0.0,0.0
14,What are some popular gift shops that offer a variety of items suitable for men?,[e0c02a01-66aa-4e58-afed-93f9e6b4ec16],"[ceb62a21-6a04-4a20-a7d5-505146c68801, 6ee713ad-1fd6-4ba7-8755-ed

In [41]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    print(f"- Expected contexts:\n")
    expected_nodes = docstore.get_nodes(row.expected_ids)
    for node in expected_nodes:
        print(node.get_content(metadata_mode=MetadataMode.LLM))

    print(f"\n- Retrieved contexts:\n")
    retrieved_nodes = docstore.get_nodes(row.retrieved_ids)
    for i, node in enumerate(retrieved_nodes):
        print(f"+ Context #{i+1}:\n\n{node.get_content(metadata_mode=MetadataMode.LLM)}\n")



============Error #1=============


Query:
What are some businesses or locations that have received poor reviews for customer service?

- Expected contexts:

review_stars: 1
biz_name: H&M
biz_address: 827-833 State St
biz_city: Santa Barbara
biz_state: CA
biz_categories: ["Women's Clothing", 'Accessories', "Children's Clothing", "Men's Clothing", 'Adult', 'Shopping', 'Fashion']

Awful.

- Retrieved contexts:

+ Context #1:

review_stars: 1
biz_name: Ardmore Pizza
biz_address: 10 Rittenhouse Pl
biz_city: Ardmore
biz_state: PA
biz_categories: ['Pizza', 'Restaurants']

This place is literally the worst customer service. They messed up my order then were very rude telling me it was my fault for not checking before the delivery guy left.... which i did and he told me to call!

+ Context #2:

review_stars: 1
biz_name: Famous Footwear
biz_address: 8522 Eager Road, Dierbergs Brentwood Point
biz_city: Brentwood
biz_state: MO
biz_categories: ['Sporting Goods', 'Fashion', 'Shoe Stores', 'Shoppi

In [42]:
query = "What are some good grocery stores in Nashville that offer curated items and unique local products?"
retrieval_results = await retriever.aretrieve(query)
for node in retrieval_results:
    display_source_node(node, source_length=1000)
    print(node.get_content(metadata_mode=MetadataMode.EMBED))
    print("\n")

Generated queries:
grocery stores nashville curated items unique local products
"I recently visited a fantastic grocery store in Nashville that specializes in curated items and unique local products. They had an amazing selection of artisanal cheeses and locally sourced produce that really stood out."


**Node ID:** c11a1dd0-0f0b-4e36-a460-fbdf55e45b25<br>**Similarity:** 0.03333333333333333<br>**Text:** Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!<br>

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

Stumbled upon this hidden gem of a store when we went to have dinner at of of the local restaurants.. 

They have some very unique items.. High quality which mean you get what you pay for! They also have several local artist/products.. 

Customer service hands down excellent!




**Node ID:** 65d8adc5-66b8-44d5-ab74-b0a2a298d7a3<br>**Similarity:** 0.03306010928961749<br>**Text:** Love this place<br>

review_stars: 5
biz_name: Charlie's Market
biz_address: 2815 E Sligh Ave
biz_city: Tampa
biz_state: FL
biz_categories: ['Food', 'Grocery', 'Convenience Stores']

Love this place




...

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

What an awesome store for dudes! My man does not like shopping, and frankly, didn't even want to go in here, but once in, he found a number of things he really liked. This is like a finely curated museum of quality manly goods. And certainly, high quality items you aren't going to find in just any ol' store.




**Node ID:** a9240351-7639-40fe-a0f8-7397b44960db<br>**Similarity:** 0.030536130536130537<br>**Text:** A unique, local men's boutique in Irvington featuring a good selection of high quality clothing, accessories, and home goods. My husband will go out of his way to visit this store.<br>

review_stars: 5
biz_name: James Dant
biz_address: 5624 E Washington St
biz_city: Indianapolis
biz_state: IN
biz_categories: ['Fashion', 'Shopping', "Men's Clothing"]

A unique, local men's boutique in Irvington featuring a good selection of high quality clothing, accessories, and home goods. My husband will go out of his way to visit this store.




**Node ID:** 578480a6-d947-4ff1-a0f1-ff14c14dd16d<br>**Similarity:** 0.03036576949620428<br>**Text:** This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.<br>

review_stars: 4
biz_name: Caviar & Bananas
biz_address: 2031 Broadway
biz_city: Nashville
biz_state: TN
biz_categories: ['Coffee & Tea', 'Restaurants', 'Wine Bars', 'Bars', 'Nightlife', 'American (Traditional)', 'Event Planning & Services', 'Food', 'Caterers', 'Breakfast & Brunch', 'Cafes', 'Diners']

This is a very interesting spot for specialty items and local items like that delicious Nashville fountain of juice. This is a great place to grab coffee, dessert or a salad / sandwich for lunch.




# Response

In [43]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [44]:
node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer()
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [48]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [49]:
question = "where to find good cold brew coffee?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

Generated queries:
cold brew coffee locations
"I recently discovered a cozy café that serves the best cold brew coffee in town. The rich flavor and smooth texture were amazing, and the atmosphere was perfect for relaxing."


**`Final Response:`** You can find good cold brew coffee at Helena Avenue Bakery in Santa Barbara, CA.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicious. Cute space ( a bit hard to find ). The green eggs and ham  breakfast biscuit is to die for.  ($10 I think) The Santa Barbra tri tip sandwich was excellent! Cold brew coffee was good.  (Review: `Q3fPo_x6xKxafAzy1hFITg`)

> Service at the register was friendly. Cold brew was sooo good as were the sandwiches we got. Will most definitely be back. (Review: `-6iyNp8Vs9kF0IRGXZvsHA`)

> Barista was fast at least. Good latte and cold brew. I should have just ordered a pastry though. (Review: `LuLzCVNRkAPjVaRVA__V2A`)

> This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit sandwich were both perfectly cooked, fresh, flavorful. We were really impressed with the quality of the cappuccino and the cold brew coffee.  (Review: `4sGHeBRdn36yOX3KszhzCA`)

**Mike's Ice Cream**

> If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee as any other coffee house you'll find. (Review: `GHAY9_Kpk-1hcq4Ehfmu1w`)

---

**`Source Node 1/5`**

**Node ID:** 7b0b5fbb-fdf7-4006-8366-cf25eff27941<br>**Similarity:** 0.7123664021492004<br>**Text:** Pricey ( a ham and cheese croissant was $5.50)  as with anywhere in Santa Barbra but very delicio...<br>**Metadata:** {'review_id': 'Q3fPo_x6xKxafAzy1hFITg', 'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 4, 'useful': 0, 'funny': 0, 'cool': 0, 'date': '2018-07-17T00:20:41', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

...

**Node ID:** 36991e23-3f96-4bcd-ac95-70c84e20e5f0<br>**Similarity:** -2.0586793422698975<br>**Text:** If it's cold, come grab a coffee instead. They offer all the great styles and flavors of coffee a...<br>**Metadata:** {'review_id': 'GHAY9_Kpk-1hcq4Ehfmu1w', 'user_id': 'HpXm_E_MRQdN_Rv4cARviA', 'business_id': 'oaboaRBUgGjbo2kfUIKDLQ', 'review_stars': 5, 'useful': 1, 'funny': 0, 'cool': 1, 'date': '2016-02-07T16:02:23', 'biz_name': "Mike's Ice Cream", 'biz_address': '129 2nd Ave N', 'biz_city': 'Nashville', 'biz_state': 'TN', 'biz_postal_code': '37201', 'biz_latitude': 36.1626492, 'biz_longitude': -86.7759733, 'biz_stars': 4.5, 'biz_review_count': 593, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'False', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': 'False', 'Caters': 'False', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'False', 'DriveThru': 'False', 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': None, 'HappyHour': None, 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'None', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': None, 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': None, 'WiFi': "'free'"}, 'biz_categories': ['Ice Cream & Frozen Yogurt', 'Coffee & Tea', 'Restaurants', 'Sandwiches', 'Food'], 'biz_hours': {'Friday': '8:0-0:0', 'Monday': '8:0-23:0', 'Saturday': '8:0-0:30', 'Sunday': '8:0-23:0', 'Thursday': '8:0-23:0', 'Tuesday': '8:0-23:0', 'Wednesday': '8:0-23:0'}}<br>

---

**`Source Node 5/5`**

**Node ID:** 17d515f7-ee7b-4ad8-ba5e-7ade77e44922<br>**Similarity:** -2.2148277759552<br>**Text:** This review is for the delicious breakfast and coffee. The Green Breakfast Bowl and the Biscuit s...<br>**Metadata:** {'review_id': '4sGHeBRdn36yOX3KszhzCA', 'user_id': 'iGhGzpTg0c3J_tuTi9TOzQ', 'business_id': 'IDtLPgUrqorrpqSLdfMhZQ', 'review_stars': 5, 'useful': 1, 'funny': 1, 'cool': 1, 'date': '2018-07-28T19:12:02', 'biz_name': 'Helena Avenue Bakery', 'biz_address': '131 Anacapa St, Ste C', 'biz_city': 'Santa Barbara', 'biz_state': 'CA', 'biz_postal_code': '93101', 'biz_latitude': 34.4144445, 'biz_longitude': -119.6906718, 'biz_stars': 4.0, 'biz_review_count': 389, 'biz_is_open': 1, 'biz_attributes': {'AcceptsInsurance': None, 'Alcohol': "u'none'", 'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}", 'BYOB': None, 'BestNights': None, 'BikeParking': 'True', 'BusinessAcceptsBitcoin': None, 'BusinessAcceptsCreditCards': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'ByAppointmentOnly': None, 'Caters': 'True', 'CoatCheck': None, 'Corkage': None, 'DogsAllowed': 'True', 'DriveThru': None, 'GoodForDancing': None, 'GoodForKids': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}", 'HappyHour': 'False', 'HasTV': 'False', 'Music': None, 'NoiseLevel': "u'average'", 'OutdoorSeating': 'True', 'RestaurantsAttire': "'casual'", 'RestaurantsDelivery': 'None', 'RestaurantsGoodForGroups': 'True', 'RestaurantsPriceRange2': '2', 'RestaurantsReservations': 'False', 'RestaurantsTableService': 'False', 'RestaurantsTakeOut': 'True', 'Smoking': None, 'WheelchairAccessible': 'True', 'WiFi': "u'no'"}, 'biz_categories': ['Food', 'Restaurants', 'Salad', 'Coffee & Tea', 'Breakfast & Brunch', 'Sandwiches', 'Bakeries'], 'biz_hours': {'Friday': '8:0-14:0', 'Monday': '0:0-0:0', 'Saturday': '8:0-14:0', 'Sunday': '8:0-14:0', 'Thursday': '8:0-14:0', 'Tuesday': '8:0-14:0', 'Wednesday': '8:0-14:0'}}<br>

{'7b0b5fbb-fdf7-4006-8366-cf25eff27941': {'review_id': 'Q3fPo_x6xKxafAzy1hFITg',
  'user_id': 'ha2Lv7WnxvdYnRsqwL1apQ',
  'business_id': 'IDtLPgUrqorrpqSLdfMhZQ',
  'review_stars': 4,
  'useful': 0,
  'funny': 0,
  'cool': 0,
  'date': '2018-07-17T00:20:41',
  'biz_name': 'Helena Avenue Bakery',
  'biz_address': '131 Anacapa St, Ste C',
  'biz_city': 'Santa Barbara',
  'biz_state': 'CA',
  'biz_postal_code': '93101',
  'biz_latitude': 34.4144445,
  'biz_longitude': -119.6906718,
  'biz_stars': 4.0,
  'biz_review_count': 389,
  'biz_is_open': 1,
  'biz_attributes': {'AcceptsInsurance': None,
   'Alcohol': "u'none'",
   'Ambience': "{'touristy': False, 'hipster': True, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': True, 'upscale': False, 'classy': False, 'casual': True}",
   'BYOB': None,
   'BestNights': None,
   'BikeParking': 'True',
   'BusinessAcceptsBitcoin': None,
   'BusinessAcceptsCreditCards': 'True',
   'BusinessParking': "{'garage': False, 'street': True, 'v

## Response Evaluation

In [50]:
from src.run.eval import ResponseEvaluator

In [51]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [52]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-08-09 17:12:08.306 | INFO     | src.run.eval.response:generate_synthetic_dataset:39 - Sampling 30 documents for response evaluation...
2024-08-09 17:12:08.314 | INFO     | src.run.eval.response:generate_synthetic_dataset:59 - Creating new response eval dataset at data/015_revamp_synthetic_eval/response_synthetic_eval_dataset.json...
2024-08-09 17:12:08.315 | INFO     | src.run.eval.response:generate_synthetic_dataset:62 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.08it/s]
2024-08-09 17:12:55.161 | INFO     | src.run.eval.response:generate_synthetic_dataset:85 - Persisting synthetic response eval dataset at data/015_revamp_synthetic_eval/response_synthetic_eval_dataset.json...


In [53]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/16 [00:00<?, ?it/s]

Generated queries:
cocktail bars welcoming atmosphere philadelphia
"I visited a fantastic cocktail bar in Philadelphia that had a really inviting vibe. The staff was friendly, and the drinks were creatively crafted, making it a perfect spot to relax with friends."
Generated queries:
ice cream shops nashville variety flavors
"I visited Jeni's Splendid Ice Creams in Nashville, and they had an incredible selection of unique flavors, from goat cheese to lavender. Definitely a must-try for ice cream lovers!"
Generated queries:
Italian restaurants Reno good customer service
"I recently dined at Bella Italia in Reno, and the service was exceptional! The staff was attentive and friendly, making the whole experience enjoyable. Highly recommend for anyone looking for great Italian food and service."
Generated queries:
sports bars indianapolis friendly staff good atmosphere
"I visited a great sports bar in Indianapolis where the staff was incredibly friendly and the atmosphere was lively. It was 

Batch processing of predictions:   0%|                                                                                                                                                                        | 0/14 [00:00<?, ?it/s]

Generated queries:
highly-rated cafes nashville great atmosphere delicious desserts
"I visited a cafe in Nashville that had an amazing atmosphere and served the most delicious desserts. The ambiance was perfect for relaxing, and the dessert menu was extensive, featuring everything from cakes to pastries."
Generated queries:
top-rated wine bars New Orleans
"I visited a fantastic wine bar in New Orleans called Bacchanal Wine, where the atmosphere is lively and the selection of wines is impressive. They also have a great outdoor space!"
Generated queries:
sports bars lively atmosphere indianapolis
"I visited a great sports bar in Indianapolis where the energy was electric! The place had multiple screens for games, and the crowd was really into it, making for an unforgettable experience."
Generated queries:
mobile phone stores excellent customer service
"I visited Tech Haven and was impressed by their customer service. The staff was knowledgeable and took the time to help me find the perfe

Batch processing of predictions: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:26<00:00,  1.90s/it]


In [54]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:22<00:00,  2.76s/it]


In [55]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.533333
mean_relevancy_score,0.900000
mean_faithfulness_score,0.933333


In [56]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are some trendy Japanese restaurants in N...,\nIf you're looking for trendy Japanese restau...,1.0,3.0,1.0,[One of my favorite restaurants in Nashville. ...
1,- What are some Italian restaurants in Reno th...,\nIt's likely that family-owned Italian restau...,1.0,3.0,0.0,[Chicken parm was good but nothing really spec...
2,What are some highly-rated wine bars in New Or...,\nIf you're looking for a wine bar experience ...,1.0,2.0,1.0,[Wow!!! One of my top three new favorite place...
3,What are some popular wine bars in New Orleans...,\nIf you're looking for a great wine bar exper...,1.0,2.0,1.0,[Nice variety of wines at very reasonable pric...
4,What are some ice cream shops that offer uniqu...,\nSome ice cream shops offer a variety of cone...,1.0,2.0,1.0,[While walking around downtown I kept seeing p...
5,- What are some coffeehouses in Edmonton that ...,\nThere are several coffeehouses in Edmonton t...,1.0,3.0,1.0,[I went there yesterday afternoon for the firs...
6,What are some highly-rated sports bars that of...,\nThere are several highly-rated sports bars t...,1.0,2.0,1.0,[Then if all of that watching gets you motivat...
7,- What are some cocktail bars in Philadelphia ...,\nIt seems that a few establishments in Philad...,1.0,2.0,1.0,[Best new bar in Philly! The menu and drinks a...
8,What are some sports bars in Indianapolis that...,\nIt seems that sports bars with friendly staf...,1.0,2.0,1.0,[Delicious food. The waitresses were so nice a...
9,What are some popular wine bars in New Orleans...,\nNew Orleans is home to a vibrant nightlife s...,1.0,3.0,1.0,[Great new restaurant in New Orleans. This pla...


#### Error Analysis

In [57]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [58]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
22,- What are some gastropubs in Indianapolis tha...,"\nUnfortunately, the provided context informat...",0.0,1.0,0.0,[Their beer is some of the best in the area an...
23,- What are some pizza places in White House th...,"\nUnfortunately, I couldn't find any informati...",0.0,2.0,1.0,[The ingredients were high quality and somethi...
25,- What are some mobile phone stores known for ...,"\nBased on the information provided, it appear...",0.0,3.0,1.0,"[Okay, so after a couple of trips to this plac..."
2,What are some highly-rated wine bars in New Or...,\nIf you're looking for a wine bar experience ...,1.0,2.0,1.0,[Wow!!! One of my top three new favorite place...
3,What are some popular wine bars in New Orleans...,\nIf you're looking for a great wine bar exper...,1.0,2.0,1.0,[Nice variety of wines at very reasonable pric...
4,What are some ice cream shops that offer uniqu...,\nSome ice cream shops offer a variety of cone...,1.0,2.0,1.0,[While walking around downtown I kept seeing p...
6,What are some highly-rated sports bars that of...,\nThere are several highly-rated sports bars t...,1.0,2.0,1.0,[Then if all of that watching gets you motivat...
7,- What are some cocktail bars in Philadelphia ...,\nIt seems that a few establishments in Philad...,1.0,2.0,1.0,[Best new bar in Philly! The menu and drinks a...
8,What are some sports bars in Indianapolis that...,\nIt seems that sports bars with friendly staf...,1.0,2.0,1.0,[Delicious food. The waitresses were so nice a...
10,- What are some popular ice cream shops in Nas...,"\nNashville has a vibrant food scene, and when...",1.0,2.0,1.0,[True Life: I'm addicted to ice cream. \n\nWhe...


In [59]:
response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
- What are some gastropubs in Indianapolis that are known for their cocktails?

Context:
Their beer is some of the best in the area and the environment they have screams of coziness. One of the most underrated breweries in Indianapolis, Bier provides one of the finer experiences in breweries, so please check out this homely feeling place soon. Shout out to the hop gummy series!

Cheers!

I went here today for my first time and recieved outstanding service, great food, and ice cold beer! The atmosphere was very welcoming as well as our server, Skye! 

McKenzie is very attentive..wont leave your beer empty not to mention her nice personality..beers are the coldest in Indianapolis..never disappointed here..

My favorite Indianapolis beers are Bier's. If you're going for the beer, you will not be disappointed. 

Answer:

Unfortunately, the provided context information does not mention gastropubs or their cocktails. However, it does mention a

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [60]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-08-09 17:20:36.560 | INFO     | src.run.eval.response:generate_curated_dataset:124 - Persisting curated response eval dataset at data/015_revamp_synthetic_eval/response_curated_eval_dataset.json...


In [61]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions:   0%|                                                                                                                                                                         | 0/1 [00:00<?, ?it/s]

Generated queries:
recommended restaurants Santa Barbara nearby
"I recently visited Santa Barbara and had an amazing meal at The Lark. The ambiance was fantastic, and the farm-to-table dishes were delicious. I also enjoyed the seafood at Brophy Bros, which had a great view of the harbor."


Batch processing of predictions: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.88s/it]


In [62]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.39s/it]


In [63]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,2.0
mean_relevancy_score,1.0
mean_faithfulness_score,1.0


In [64]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's often a good idea to explore local eate...,1.0,2.0,1.0,[Finally some good food in downtown Santa Barb...


#### Answers for target questions

In [65]:
from IPython.display import Markdown

In [66]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [67]:
for question, expected_answer in MANUAL_EVAL_QA:
    display(Markdown(f"\n\n### Question: {question}\n"))
    response = query_engine.query(question)
    display_response(response)



### Question: What are the recommended restaurants nearby Santa Barbara?


Generated queries:
recommended restaurants Santa Barbara
"I recently visited Santa Barbara and tried a few restaurants that were highly recommended. The seafood at The Boathouse was fresh and delicious, and I loved the ambiance at Finch & Fork. Definitely worth checking out!"


**`Final Response:`** It's always exciting to discover new restaurants in a charming city like Santa Barbara. If you're looking for a great breakfast spot, consider visiting a bakery that's located in the trendy FUNK ZONE, close to the wharf. This bakery is a must-visit for its delicious offerings and convenient downtown location.


#### Referenced Paragraphs
**Helena Avenue Bakery**

> Finally some good food in downtown Santa Barbara.  Everything on the menu is great and could not recommend more.  (Review: `8qeD6EhCxUEBj1dO4J4KfA`)

> If you're a tourist and spending the weekend in Santa Barbara, I'd recommend to give this place a go for sure. If you're a local, then not so much.  (Review: `Mqh2anPpP_vvLtqCAk_1Xw`)

> Helena Avenue Bakery is a must-visit for those looking for a great breakfast place in Santa Barbara. Not only is it in Downtown Santa Barbara, but it is located in the trendy FUNK ZONE close to the wharf.  (Review: `MvkzsY6BH6Fb5AABgLyAhg`)

> Lol and maybe reiterate it since they seemed to struggle with that in our order (Review: `HKLjinZDrr7WGqxMfx74Cg`)

> How lucky you folks are in Santa Barbara! (Review: `m0xPYpM8UnVXmbRNOTxgjQ`)

### Error Analysis

In [68]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [69]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are the recommended restaurants nearby Sa...,\nIt's often a good idea to explore local eate...,1.0,2.0,1.0,[Finally some good food in downtown Santa Barb...


In [70]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [71]:
from src.run.utils import parse_collect_log, flatten_dict

In [72]:
collect_info = parse_collect_log(collect_fp)

In [73]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    try:
        response_evaluator.log_to_mlflow(
            cfg,
            'synthetic',
            response_synthetic_mean_scores_df,
            response_synthetic_deep_eval_df
        )
        response_evaluator.log_to_mlflow(
            cfg,
            'curated',
            response_curated_mean_scores_df,
            response_curated_deep_eval_df
        )
    except:
        ...

2024-08-09 17:20:45.828 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-08-09 17:20:45.860 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/final-project/review-rec-bot/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-08-09 17:20:45.894 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-08-09 17:20:46.340 | INFO     | __main

# Clean up

In [74]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()

# Archive